In [1]:
import pandas as pd
import glob
import os
import numpy as np

In [2]:
county = ['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER', 'TUOLUMNE', 'MENDOCINO', 'KERN', 'TULARE']

In [3]:
path1 = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
file_type1 = '.xlsx'
load_it1 = glob.glob(path1 + file_type1)

df1 = pd.read_excel(max(load_it1, key=os.path.getctime))

In [4]:
# read in South Central Smart sheeets tracker
path_s = r"C:\Users\jacque.trahan\Downloads\South Central Division 2021 Debris Removal Tracker*"
file_type_s = '.xlsx'
load_it_s = glob.glob(path_s + file_type_s)

df_s = pd.read_excel(max(load_it_s, key=os.path.getctime), parse_dates=True)

### Concat central and south cental files

In [5]:
df_both = pd.concat([df1, df_s])

In [6]:
path2 = r"C:\Users\jacque.trahan\Downloads\BIO*"
file_type2 = ".csv"
load_it2 = glob.glob(path2 + file_type2)

df2 = pd.read_csv(max(load_it2, key=os.path.getctime))

In [7]:
path3 = r"C:\Users\jacque.trahan\Downloads\Arch*"
file_type3 = ".csv"
load_it3 = glob.glob(path3 + file_type2)

df3 = pd.read_csv(max(load_it3, key=os.path.getctime))

In [8]:
# df1 = pd.read_excel('Central Branch  trackers/Central Division 2021 Debris Removal Tracker.xlsx')
# df2 = pd.read_csv('../Power BI CSVs/BIO.csv')
# df3 = pd.read_csv('../Power BI CSVs/Arch.csv')

In [9]:
smartsheets = df_both.copy()
bio = df2.copy()
arch = df3.copy()

# smart sheet columns needed check:
APN_ROW Segment 'Arch Monitor Needed','Bio Monitor Needed'

In [10]:
# smartsheets.info(verbose=True)

In [11]:
# select columns we need
smartsheets = smartsheets[['APN_ROW Segment','Street #','Street Name', 'Structural Status','County',
                           'Arch Monitor Needed','Arch Assessment',
                           'Bio Monitor Needed','Bio Assessment']]

In [12]:
# smartsheets['Arch Monitor Needed'].value_counts()

In [13]:
smartsheets['Arch Assessment'] = pd.to_datetime(smartsheets['Arch Assessment']).dt.date
smartsheets['Bio Assessment'] = pd.to_datetime(smartsheets['Bio Assessment']).dt.date

In [14]:
smartsheets.rename(columns={'Arch Monitor Needed': 'Arch Monitor Needed_SS','Arch Assessment':'Arch Assessment_SS',
                            'Bio Monitor Needed':'Bio Monitor Needed_SS','Bio Assessment':'Bio Assessment_SS'}, inplace=True)

# BIO columns needed:
APN , Biological monitoring recommended?

In [15]:
# select columns we need
bio = bio[['APN', 'Biological monitoring recommended?','County', 'Observation Date']]

In [16]:
# get just for central dixie fire
bio = bio[bio['County'].isin(county)]

### Get BIO APN count in Central

In [17]:
# pd.set_option('display.max_rows', 400)

In [18]:
bio[['APN']].value_counts()

APN        
041-940-020    16
028-270-048    12
028-310-045     6
028-360-016     6
041-830-006     4
               ..
041-290-014     1
041-290-015     1
041-381-002     1
041-381-005     1
999-999-999     1
Length: 962, dtype: int64

In [19]:
bio[['APN']].value_counts().to_excel('Bio APN Counts central.xlsx')

In [20]:
# bio['Biological monitoring recommended?'].value_counts()

In [21]:
bio['Observation Date'] = pd.to_datetime(bio['Observation Date']).dt.date

In [22]:
# change column name values to match smart sheets
bio['Biological monitoring recommended?'] = bio['Biological monitoring recommended?'].str.replace('No Debris, No Tree', 'No Bio Monitoring Needed for Debris or Trees')
bio['Biological monitoring recommended?'] =bio['Biological monitoring recommended?'].str.replace('Yes Debris, Yes Tree', 'Bio Monitoring Needed for Debris AND Trees')
bio['Biological monitoring recommended?'] =bio['Biological monitoring recommended?'].str.replace('No Debris, Yes Tree', 'Bio Monitoring Needed for Trees, not Debris')
bio['Biological monitoring recommended?'] =bio['Biological monitoring recommended?'].str.replace('Yes Debris, No Tree','Bio Monitoring Needed for Debris, not Trees')

In [23]:
bio.rename(columns={'Observation Date':'Observation Date_BIO'}, inplace=True)

# ARCH columns needed:
APN , Archaeological Resources Present?

In [24]:
# select columns we need
arch = arch[['APN', 'Monitoring Recommendation','Date', 'County']]

In [25]:
# get just for central dixie fire
arch = arch[arch['County'].isin(county)]

### Get ARCH APN count for central

In [26]:
arch['APN'].value_counts().to_excel('ARCH APN count central.xlsx')

In [27]:
# arch['Monitoring Recommendation'].value_counts()

In [28]:
# arch['Monitoring Recommendation'].value_counts()

In [29]:
# formate dates 
arch['Date'] = pd.to_datetime(arch['Date']).dt.date

In [30]:
# change column name values to match smart sheets
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('No Debris, No Tree', 'No Arch Monitoring Needed for Debris or Trees')
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('Yes Debris, Yes Tree', 'Arch Monitoring Needed for Debris AND Trees')
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('No Debris, Yes Tree', 'Arch Monitoring Needed for Trees, not Debris')
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('Yes Debris, No Tree', 'Arch Monitoring Needed for Debris, not Trees')

In [31]:
arch.rename(columns={'Monitoring Recommendation':'Monitoring Recommendation_ARCH', 'Date': 'Date_ARCH'},inplace=True)

# Merge Data bases
* smartsheets and BIO
* smasrtsheet and Arch

# Smartsheet and BIO

In [32]:
# Merge Smartsheets and BIO

smartsheets_bio = smartsheets.merge(bio,
                          left_on='APN_ROW Segment',
                          right_on='APN',
                          how='outer',
                          suffixes=('_SS',"_ARCH"))

In [33]:
smartsheets_bio.rename(columns={'APN':'APN_BIO'}, inplace=True)

In [34]:
# smartsheets_bio.head(0)

In [35]:
# smartsheets_bio.info()

In [36]:
smartsheets_bio['Bio Monitor Needed_SS'] = smartsheets_bio['Bio Monitor Needed_SS'].fillna("0")
smartsheets_bio['Biological monitoring recommended?'] = smartsheets_bio['Biological monitoring recommended?'].fillna('0')
smartsheets_bio['Observation Date_BIO'] = smartsheets_bio['Observation Date_BIO'].fillna("0")
smartsheets_bio['Bio Assessment_SS'] = smartsheets_bio['Bio Assessment_SS'].fillna("0")

# Smartsheet and Arch

In [37]:
smartsheets_ARCH = smartsheets.merge(arch,
                    left_on='APN_ROW Segment',
                    right_on='APN',
                    how='outer',
                    suffixes=('_SS',"_ARCH"))

In [38]:
smartsheets_ARCH.rename(columns={'APN':'APN_ARCH'}, inplace=True)

In [39]:
# smartsheets_ARCH.head(0)

In [40]:
# smartsheets_ARCH['Date_ARCH'].dtypes

In [41]:
smartsheets_ARCH['Arch Monitor Needed_SS'] = smartsheets_ARCH['Arch Monitor Needed_SS'].fillna("0")
smartsheets_ARCH['Monitoring Recommendation_ARCH'] = smartsheets_ARCH['Monitoring Recommendation_ARCH'].fillna('0')
smartsheets_ARCH['Date_ARCH'] = smartsheets_ARCH['Date_ARCH'].fillna("0")
smartsheets_ARCH['Arch Assessment_SS'] = smartsheets_ARCH['Arch Assessment_SS'].fillna("0")

# Set up column order for each Data base

In [42]:
# ARch and smart sheets
smartsheets_ARCH = smartsheets_ARCH[['APN_ROW Segment', 'APN_ARCH','Street #','Street Name', 'Structural Status','County_SS',
       'Arch Monitor Needed_SS', 'Monitoring Recommendation_ARCH',
                                     'Arch Assessment_SS','Date_ARCH']]

In [43]:
# BIO and smasrt sheets
smartsheets_bio = smartsheets_bio[['APN_ROW Segment', 'APN_BIO','Street #','Street Name', 'Structural Status','County_SS',
                                   'Bio Monitor Needed_SS', 'Biological monitoring recommended?',
                                   'Bio Assessment_SS','Observation Date_BIO']]

# Compare columns in smart sheet to other databases

In [44]:
def compare_cols(df, comp1, comp2):
    if df[comp1] == df[comp2]:
        return 1
    else:
        return 0

# compart ARCH monitor needed to smartsheet

In [45]:
smartsheets_ARCH.insert(smartsheets_ARCH.columns.get_loc('Monitoring Recommendation_ARCH') + 1, "ARC Check",
            smartsheets_ARCH.apply(compare_cols, comp1='Arch Monitor Needed_SS', comp2='Monitoring Recommendation_ARCH',axis=1))

# comapre dates for ARCH and smartsheets dates

In [46]:
smartsheets_ARCH.insert(smartsheets_ARCH.columns.get_loc('Date_ARCH') + 1, 'Check Arch Dates',
            smartsheets_ARCH.apply(compare_cols, comp1='Arch Assessment_SS', comp2='Date_ARCH', axis=1))

# check BIO and Smartsheet dates

In [47]:
smartsheets_bio.insert(smartsheets_bio.columns.get_loc('Observation Date_BIO') + 1, 'Check BIO Dates',
            smartsheets_bio.apply(compare_cols, comp1='Bio Assessment_SS', comp2='Observation Date_BIO',axis=1))

# Compare BIO monitor needed to  smartsheets 

In [48]:
smartsheets_bio.insert(smartsheets_bio.columns.get_loc('Biological monitoring recommended?') + 1, 'BIO Check',
                                  smartsheets_bio.apply(compare_cols, comp1='Bio Monitor Needed_SS',
                                              comp2='Biological monitoring recommended?',axis=1))

# highligh the columns that are comparing values

In [49]:
df_style_BIO = smartsheets_bio.style.set_properties(**{'background-color': '#ffffb3'},
                          subset=['BIO Check', 'Check BIO Dates'])

In [50]:
df_style_ARCH = smartsheets_ARCH.style.set_properties(**{'background-color': '#ffffb3'},
                                                    subset=["ARC Check", 'Check Arch Dates'])

In [51]:
df_style_BIO.to_excel('BIO checks.xlsx')

In [52]:
df_style_ARCH.to_excel('ARCH check.xlsx')